# Scoring Function
A scoring function is a function that takes as input a number of objective values and returns an aggregated desirability score.  
The scoring function is dynamically constructed based on a scoring profile, defining the aggregation algorithm, and, for each objective, its desirability function and the required parameters.



## Code Examples

In [30]:
# import the library and print the version
import pumas
print(pumas.__version__)

0.0.0


### Prepare a Scoring Profile


In [31]:
scoring_profile_dict = {
    "objectives": [
        {
            "name": "quality",
            "desirability_function": {
                "name": "sigmoid",
                "parameters": {
                    "low": 1.0,
                    "high": 10.0,
                    "k": 0.1,
                    "shift": 0.0,
                    "base": 10.0,
                },
            },
            "weight": 1.0,
            "value_type": "float",
            "kind": "numerical",
        },
        {
            "name": "efficiency",
            "desirability_function": {
                "name": "multistep",
                "parameters": {
                    "coordinates": [(0.0, 0.0), (0.5, 0.7), (1.0, 1.0)],
                },
            },
            "weight": 2.0,
            "value_type": "float",
            "kind": "numerical",
        },
        {
            "name": "cost",
            "desirability_function": {
                "name": "sigmoid",
                "parameters": {
                    "low": 20.0,
                    "high": 80.0,
                    "k": -0.5,
                    "shift": 0.0,
                    "base": 10.0,
                },
            },
            "weight": 3.0,
            "value_type": "float",
            "kind": "numerical",
        },
    ],
    "aggregation_function": {
        "name": "geometric_mean",
        "parameters": {},
    },
}

In [32]:
from pumas.scoring_profile.scoring_profile import Profile
scoring_profile = Profile.model_validate(scoring_profile_dict)

### Prepare the data

In [33]:
data_payload = {
    "A":{ "quality": 1.0, "cost": 30.0, "efficiency": 0.7},
    "B":{ "quality": 5.0, "cost": 40.0, "efficiency": 0.2},
    "C":{ "quality": 3.0, "cost": 10.0, "efficiency": 0.9},
    "D":{ "quality": 6.0, "cost": 60.0, "efficiency": 0.4},
    "E":{ "quality": 2.0, "cost": 80.0, "efficiency": 0.5},
}

### Initialize the Scoring Function

In [34]:
from pumas.scoring_framework.scoring_function import ScoringFunction
scoring_function = ScoringFunction(
        profile=scoring_profile,
    )

### Use the Scoring Function on Data

In [35]:
result = scoring_function.compute(data=data_payload.get("A"))
print(result)

{'aggregated_score': 0.7301646955230212, 'desirability_scores': {'quality': 0.2402530733520421, 'efficiency': 0.82, 'cost': 0.9789100228828135}}


In [36]:
results_output = {uid:scoring_function.compute(data=data) for uid, data in data_payload.items()}
from pprint import pprint
pprint(results_output, indent=2)

{ 'A': { 'aggregated_score': 0.7301646955230212,
         'desirability_scores': { 'cost': 0.9789100228828135,
                                  'efficiency': 0.82,
                                  'quality': 0.2402530733520421}},
  'B': { 'aggregated_score': 0.538307028943963,
         'desirability_scores': { 'cost': 0.8720068922233212,
                                  'efficiency': 0.27999999999999997,
                                  'quality': 0.4680631904645068}},
  'C': { 'aggregated_score': 0.8203176039237978,
         'desirability_scores': { 'cost': 0.999536056460154,
                                  'efficiency': 0.94,
                                  'quality': 0.34533538970278355}},
  'D': { 'aggregated_score': 0.26543975946499104,
         'desirability_scores': { 'cost': 0.1279931077766788,
                                  'efficiency': 0.5599999999999999,
                                  'quality': 0.5319368095354932}},
  'E': { 'aggregated_score': 0.040558016372